# Relax Inc. Take-Home Challenge

Defining an "adopted user" as a user who has logged into the product on three separate days in at least one seven­day period, identify which factors predict future user adoption.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from pandasql import sqldf
from sqlalchemy import create_engine 
from sqlalchemy import MetaData, Table
import sqlite3
import time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import tree
import warnings
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

In [2]:
# Read the csv file
df1 = pd.read_csv('takehome_user_engagement.csv', encoding='latin-1')
df2 = pd.read_csv('takehome_users.csv', encoding = 'latin-1')

In [3]:
# See the columns of the fisrt data frame
df1.columns

Index(['time_stamp', 'user_id', 'visited'], dtype='object')

In [4]:
# See the columns of the second data frame
df2.columns

Index(['object_id', 'creation_time', 'name', 'email', 'creation_source',
       'last_session_creation_time', 'opted_in_to_mailing_list',
       'enabled_for_marketing_drip', 'org_id', 'invited_by_user_id'],
      dtype='object')

In [5]:
#Shape of data frames
print('First data frame Rows, columns {}'.format(df1.shape))
print('Second data frame Rows, columns {}'.format(df2.shape))

First data frame Rows, columns (207917, 3)
Second data frame Rows, columns (12000, 10)


In [6]:
# See the information of the data frame
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null object
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [7]:
# See the information of the data frame
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
object_id                     12000 non-null int64
creation_time                 12000 non-null object
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            6417 non-null float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [8]:
# To datetime 
df1['time_stamp'] = pd.to_datetime(df1['time_stamp'], format = '%Y-%m-%d')
df2['creation_time'] = pd.to_datetime(df2['creation_time'], format = '%Y-%m-%d')
# Convert last_session_creation_time to float
df2['last_session_creation_time'] = df2['last_session_creation_time'].astype(float)

In [9]:
# See the null values
for col in df2.columns:
    print(col)
    print(df2[col].isna().sum())

object_id
0
creation_time
0
name
0
email
0
creation_source
0
last_session_creation_time
3177
opted_in_to_mailing_list
0
enabled_for_marketing_drip
0
org_id
0
invited_by_user_id
5583


In [10]:
#Fill null values
df2['last_session_creation_time'] = df2['last_session_creation_time'].replace(np.nan, 'Not Registered')

In [11]:
# Rename the columns to future merge
df2.rename(columns = {'object_id':'user_id'}, inplace = True)

In [12]:
# Merge both columns
df = pd.merge(df1, df2, how = 'inner', on = 'user_id')

In [13]:
# Set the time_stamp index to rsample
df.set_index('time_stamp', inplace = True)

In [14]:
df.head()

,user_id,visited,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
time_stamp,,,,,,,,,,,
2014-04-22 03:53:30,1,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.39814e+09,1,0,11,10803.0
2013-11-15 03:45:04,2,1,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.39624e+09,0,0,1,316.0
2013-11-29 03:45:04,2,1,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.39624e+09,0,0,1,316.0
2013-12-09 03:45:04,2,1,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.39624e+09,0,0,1,316.0
2013-12-25 03:45:04,2,1,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.39624e+09,0,0,1,316.0


### General Insights

##### We can get some general insights by doing some querying

In [15]:
# Define a function to query
def pysqldf(q):
    return sqldf(q, globals())

In [16]:
# Get the Users that invited more users
print(pysqldf('''SELECT invited_by_user_id, COUNT(invited_by_user_id) AS count, name FROM df 
                GROUP BY invited_by_user_id 
                ORDER BY count DESC
                LIMIT 20
            '''))

    invited_by_user_id  count              name
0               4019.0    918   Bergmann Sabine
1               7816.0    903         Field Eve
2               3623.0    744        Lack Tyson
3               7358.0    712     Terrell Jesus
4               6312.0    705       Stone Elise
5               4276.0    644      Adler Marcel
6                879.0    630    Jones Margaret
7               2175.0    621      Santos Bruna
8               3269.0    605     Outtrim Gemma
9               8068.0    598  Schuster Dominik
10              1266.0    566       Watts Ellis
11             11960.0    559        Bak Kasper
12              6560.0    550      Sawyer Lucas
13              7210.0    549       Barbosa Kai
14               454.0    548      Egger Daniel
15              7529.0    545   Andrade Katrina
16              7792.0    538     Grunewald Tom
17              3023.0    538     Castro Rebeca
18               510.0    538        Pugh Naomi
19              5910.0    531       Cove

It is very predominant to have no users_id invited by another user. Maybe there should be ways to work and have benefits to make people invite users.

In [17]:
# Get the Users with more visitings
print(pysqldf('''SELECT COUNT(visited) as visitings, user_id, name
                FROM df
                GROUP BY user_id               
                HAVING visitings > 500              
                ORDER BY visitings DESC
'''))

    visitings  user_id                name
0         606     3623        Melo Vitória
1         600      906      Becker Jessika
2         593     1811    Carvalho Eduardo
3         590     7590     Ferreira Rebeca
4         585     8068         McCoy Donna
5         581     9402         Molloy Alex
6         580     4022       Schmidt Johan
7         566     3269       Outtrim Gemma
8         547     9312  Kranewitter Elijah
9         545     7210     Herzog Benjamin
10        541     8029    Oliveira Isabela
11        538     2474       Grunewald Tom
12        526     2658   Fernandes Lavinia
13        521     8280     Hanson Mohammed
14        521       69        Bowen Alisha
15        520     5297      Follett Joseph
16        519     4019          Kjær Elias
17        517     9083         Licon Karen
18        511     2519        Howitt Aidan
19        509     4263      Schultz Mikkel
20        502     3226        Schou Laerke
21        501    10599       Azevedo Tiago


In [18]:
# Get the organizations with more visitings
print(pysqldf('''SELECT org_id, COUNT(visited) AS visitings
                FROM df
                GROUP BY org_id
                ORDER BY visitings DESC
                LIMIT 20
'''))

    org_id  visitings
0        1       2298
1       13       2098
2       62       2072
3        4       2064
4        2       1967
5        7       1922
6        3       1868
7      225       1579
8       16       1524
9      161       1491
10       8       1461
11      31       1457
12     218       1447
13     235       1445
14     117       1441
15      78       1397
16       0       1390
17     108       1369
18      44       1361
19     160       1327


In [19]:
# Get the organizations with more invited numbers
print(pysqldf('''SELECT org_id, COUNT(invited_by_user_id) AS number_invited
                FROM df
                GROUP BY org_id
                ORDER BY number_invited DESC
                LIMIT 20
'''))

    org_id  number_invited
0       13            1533
1       62            1383
2      106            1301
3        4            1293
4      117            1259
5        3            1215
6      317            1157
7        8            1125
8      185            1031
9       69             998
10     218             994
11      70             994
12     320             993
13     305             969
14      85             953
15      64             944
16     224             929
17      61             912
18     368             910
19      42             866


In [20]:
# Get organizations with more number of users
print(pysqldf('''SELECT org_id, COUNT(user_id) AS number_users
                FROM df
                GROUP BY org_id
                ORDER BY number_users DESC
                LIMIT 20
'''))

    org_id  number_users
0        1          2298
1       13          2098
2       62          2072
3        4          2064
4        2          1967
5        7          1922
6        3          1868
7      225          1579
8       16          1524
9      161          1491
10       8          1461
11      31          1457
12     218          1447
13     235          1445
14     117          1441
15      78          1397
16       0          1390
17     108          1369
18      44          1361
19     160          1327


In [21]:
# Get what is the higher creation source
print(pysqldf('''SELECT creation_source, COUNT(creation_source) AS number_creations
                FROM df
                GROUP BY creation_source
                ORDER BY number_creations DESC

'''))

      creation_source  number_creations
0          ORG_INVITE             70091
1        GUEST_INVITE             46796
2              SIGNUP             37545
3  SIGNUP_GOOGLE_AUTH             34263
4   PERSONAL_PROJECTS             19222


In [22]:
# See if the opted in to mailing list has any influence on the number of visitings
print(pysqldf('''SELECT opted_in_to_mailing_list, SUM(visited) AS number_visitings
                FROM df
                GROUP BY opted_in_to_mailing_list
'''))

   opted_in_to_mailing_list  number_visitings
0                         0            154246
1                         1             53671


In [23]:
# Get if the enabled for marketing drip has infuence on the number of visitings
print(pysqldf('''SELECT enabled_for_marketing_drip, SUM(visited) AS number_visitings
                FROM df
                GROUP BY enabled_for_marketing_drip
'''))

   enabled_for_marketing_drip  number_visitings
0                           0            175787
1                           1             32130


#### As general insights, we can see that maybe the org_invite source is the source with the highest number of creations and looks to be the most important one. Looks like depending on the source, they could have higher number of creations and there is still a way to improve signups by more marketing. 


#### It is interesting that organization 1 has the most number of users, but they are not in the top 20 of number of visitings

## Modeling

In [24]:
# Create a data frame with the active users
df1_resampled = pd.DataFrame(df1.set_index('time_stamp').groupby('user_id')['visited'].resample('D').count())

In [25]:
df1_resampled.head()

visited
user_id time_stamp         
1       2014-04-22        1
2       2013-11-15        1
        2013-11-16        0
        2013-11-17        0
        2013-11-18        0

In [26]:
# Sum number of visits for seven-rolling-day periods.
df1_resampled = df1_resampled.rolling(window=7, min_periods=1).sum()
# Compute max number of visits for each user for seven-rolling-day periods.
df1_resampled2 = df1_resampled.groupby('user_id')[['visited']].max()
df1_resampled2.head()

,visited
user_id,
1,1.0
2,3.0
3,2.0
4,3.0
5,4.0


In [27]:
# Add a column 'adopted'
df1_resampled2['adopted_user'] = [1 if x >= 3.0 else 0 for x in df1_resampled2['visited']] 

In [28]:
# Merge the data frames
df = pd.merge(df1_resampled2, df2, how = 'inner', on = 'user_id')

In [29]:
# See how the new data frame looks like
df.head()

,user_id,visited,adopted_user,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,1.0,0,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.39814e+09,1,0,11,10803.0
1,2,3.0,1,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.39624e+09,0,0,1,316.0
2,3,2.0,0,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.36373e+09,0,0,94,1525.0
3,4,3.0,1,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.36921e+09,0,0,1,5151.0
4,5,4.0,1,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.35885e+09,0,0,193,5240.0


In [30]:
# Get the columns I need to make the modeling part
df = df.drop(columns = ['visited', 'name', 'email', 'creation_time', 'invited_by_user_id'], axis = 0)

In [31]:
# Change last_session_creation_type to float
df['last_session_creation_time'] = df['last_session_creation_time'].astype(float)

In [32]:
#Establish categorical features
org_dummies = pd.get_dummies(df['org_id'], prefix = 'org')
creation_source_dummies = pd.get_dummies(df['creation_source'], prefix = 'org')

In [33]:
# Create the data frame of categorical features
df_dummies = df.drop(columns = ['org_id','creation_source', 'user_id', 'last_session_creation_time'], axis = 1)
df_dummies = pd.concat([df_dummies, org_dummies,creation_source_dummies], axis = 1)

## Modeling

#### Classification Problem. We are going to make a logistic regression to see the importance of each feature

#### Create the arrays

In [34]:
# Create the arrays
X = df_dummies.drop(columns = 'adopted_user', axis = 1).values
y = df_dummies['adopted_user'].values

In [35]:
# Divide into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

### Logistic Regression

In [36]:
# Run the model
warnings.filterwarnings('ignore')
model = LogisticRegression()
model_fit = model.fit(X_train, y_train)
y_pred = model_fit.predict(X_test)

In [37]:
# Confusion Matrix
print(confusion_matrix(y_test, y_pred))

[[   0  244]
 [   0 2403]]


In [38]:
# Classification Matrix
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       244
           1       0.91      1.00      0.95      2403

    accuracy                           0.91      2647
   macro avg       0.45      0.50      0.48      2647
weighted avg       0.82      0.91      0.86      2647



From the data we can imply that there are very low 0, so the model is not really able to predict this outcome very well, so it basically predicts all of the outcomes 0 which gives you very bad score. To increase this, we are going to resample and increase the 0 number to see if we can get better outcomes. We are trying to improve outcomes by incrasing the 0 percentage from 10% to 25% and see how it goes.

## Second Model

In [39]:
#Rearrange the adopted user, so we will have more zero and see if we can create a more reliable model
df_user = df.loc[df['adopted_user'] == 1].reset_index(drop = True)
df_user = df_user.drop(columns = 'adopted_user')
df_no_user = df.loc[df['adopted_user'] == 0].reset_index(drop = True)

In [40]:
#Establish the new outcomes from just the adopted user data frame
change_outcome = pd.DataFrame({'adopted_user':[0 for x in range(0,4000)]})
no_change_outcome = pd.DataFrame({'adopted_user':[1 for x in range(0,8015-4000)]})

In [41]:
# Rearrange data frame
df_user_rearranged = change_outcome.append(no_change_outcome).reset_index(drop = True)
df_user = pd.concat([df_user, df_user_rearranged], axis = 1)

In [42]:
# New data frame
df = pd.concat([df_user, df_no_user], axis = 0)

In [43]:
# Repeat the process of categorical features
org_dummies = pd.get_dummies(df['org_id'], prefix = 'org')
creation_source_dummies = pd.get_dummies(df['creation_source'], prefix = 'creation')

In [44]:
# New data frame
df_dummies = df.drop(columns = ['org_id','creation_source', 'user_id', 'last_session_creation_time'], axis = 1)
df_dummies = pd.concat([df_dummies, org_dummies, creation_source_dummies], axis = 1)

In [45]:
# Create the arrays
X = df_dummies.drop(columns = 'adopted_user', axis = 1).values
y = df_dummies['adopted_user'].values

In [46]:
# Divide into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [47]:
# Create the model
model = LogisticRegression()
model_fit = model.fit(X_train, y_train)
y_pred = model_fit.predict(X_test)

In [48]:
# Confussion Matrix
print(confusion_matrix(y_test, y_pred))

[[960 493]
 [813 381]]


In [49]:
# Classification Report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.54      0.66      0.60      1453
           1       0.44      0.32      0.37      1194

    accuracy                           0.51      2647
   macro avg       0.49      0.49      0.48      2647
weighted avg       0.49      0.51      0.49      2647



In [50]:
# Create values with the columns
columns = list(df_dummies.columns)[1:]

In [51]:
df_coef = pd.DataFrame()
df_coef['Variable'] = columns
df_coef['Coefficient'] = model_fit.coef_.T

In [52]:
# See sorted values
df_coef.sort_values(by = 'Coefficient', ascending = False).head(n = 15)

,Variable,Coefficient
335,org_333,1.220585
189,org_187,1.212508
204,org_202,1.069758
172,org_170,0.988544
395,org_393,0.931008
184,org_182,0.913662
321,org_319,0.904509
197,org_195,0.901315
236,org_234,0.865921
74,org_72,0.859385


## Final Conclusion

##### Based on the results, we can conclude organization is the most important variable that defines if an user is adopted or not.
##### The last model would be the best one to predict if an user would be adapted or not based on the organization id.
##### Besides, if they can increase the feature importance of the organizations with more vistings, and acquisitions, this will also help the adopted user rate